# Exporting and converting any UNDL records 

## Objective

This notebook provides easy to use script for library staff without in-depth knowledge of Python. The objective of this yearbook is to export MARC XML data using UN Digital Library Search API.

If you want to understand and learn the code, consult [UNDL data and metadata - Downloading MARC XML records with the search API](learn_undlmd_marcxml_undl_search_api.ipynb)

## How does it work?

⏩ Run the next block of code

✅ Run the next block of code is optional

🆎 Input is required before running the next block of code

## 1. Prerequisite
### 1.1 Knowledge and skills 

* Know how to execute Jupyter Notebooks
* Understand the basic of Python: lists, dictionaries, functions.
* Be able to update variables in Python scripts
* Understand the basic of the Library metadata
* Being proficient at searching the UN Digital Library

### 1.2 Environment

You need the following programme and folders to run this python script locally on your computer:

* Python 3
* A root folder with the name of your choice. This root folder must have the following subfolders:
  * /data/
  * /notebook/
  * /exclude/ : the API key for the UN Digital library is stored in this folder under the name `key.json`. Do not share this key with anyone else.
  * /src/ : this folder contains specific python libraries developped by IAMU: `unlibmd.py`
* This notebook need to be stored in a /notebook/ folder.
* a /data/ folder where the data and reports will be stored: 

### ⏩ 1.3 Python Libraries
* `unlibmd`: local library focusing on manipulation the library metadata.
* Pandas: library used to store and manipulate data in tabular format (csv-like)

⏩ Import `unlibmd` and `pandas`

In [44]:
import importlib
import sys
sys.path.append('../src')
import unlibmd as md
importlib.reload(md)
import pandas as pd

## 2. UNDL Search API - Get and save records in MARC XML

### 🆎 2.1 Update the earch parameters and API key

* format: always use xml
* c: collection - if you don't apply a collection the search will be on all collections including authorities. To exclude authority records use "Resource Type". To limit to another collection (i.e. General Assembly), use the collection name as it appears in the UNDL collections tree.
* p: search pattern - the search expression you enter in the UNDL search box
* f: search index (author, documentsymbol, etc.)

Store the search parameters in a variable `parameters`

In [45]:
# INPUT - Update the parameters if necessary
parameters = {
    "format": 'xml',
    "c": 'Documents and Publications', # Collection
    "p": '856:digitallibrary AND 079:%' # Search box
}

✅ 🆎 If you have your key stored in a variable key, update the arguments before calling the function:

    * use `md.get_records_xml(parameters)` if you have your key stored in `../exclude/keys.json`.
    * use `md.get_records_xml(parameters, key)` if you don't sore your key in a variable `key`.

In [46]:
# OPTONAL INPUT
# key = 'key_to_enter_here'

### ⏩ 2.2 Get the MARC XML records and store the result in a variable `all_records`.

In [47]:
# RUN
xml_tree = md.get_records_xml(parameters)

Total nb. of records: 5872
Nb. of records processed: 1000
Nb. of records processed: 2000
Nb. of records processed: 3000
Nb. of records processed: 4000
Nb. of records processed: 5000


In [5]:
# RUN AND OBSERVE
# Check that the XML tree has the right number of records
root = xml_tree.getroot()
len(root.findall("record"))

5872

### ✅ 2.3 Save the records in XML (Optional)

🆎 Un-comment the code
🆎 Set the XML file path using the following structure `../folder/file_name.xml`
* `folder`: name of the folder where you store the export (i.e. data)
* `test.xml`: name of the file (i.e. voting_records.xml)

In [6]:
# OPTIONAL - UNCOMMENT AND UPDATE
# xml_file_path = "../folder/test.xml"
# all_records.write(xml_file_path, encoding="utf-8")

## 3. Extract MARC fields and subfields from MARC XML

### 🆎 3.1 Define which MARC elements you want to extract

To define the MARC elements you want to extract update `record_map`. This dicitonary is structure as follow:
* "field": MARC field number
* "element": The element you want to extract. Use "field" to extract the full MARC field. Use "subfield" to extract one subfield only.
* "code": The subfield code ('a','b', 'c'). Use `None` if you want to extract the full field.
* "ind1": Use only if you need to restrict the seleciton of the metadata elment by first indictator.
* "name": Column header name in the spreadsheet

In [26]:
# INPUT - Update the record_map dictionary as needed
record_map = [
    # No need to retrieve 001 with the undl_id, this is done automatically
    {   
    "field": "035",
    "element": "subfield",
    "code": "a",
    "ind1": None,
    "name": "035__a", # Change to human readable label if you wish
    },
     {   
    "field": "020",
    "element": "subfield",
    "code": "a",
    "ind1": None,
    "name": "isbn(020__a)", # Change to human readable label if you wish
    },
     {   
    "field": "079",
    "element": "subfield",
    "code": "a",
    "ind1": None,
    "name": "sales_number(079__a)", # Change to human readable label if you wish
    },
    {   
    "field": "191",
    "element": "subfield",
    "code": "a",
    "ind1": None,
    "name": "document_symbol(191__a)", # Change to human readable label if you wish
    },
    {   
    "field": "245",
    "element": "field",
    "code": None,
    "ind1": None,
    "name": "title(245)",
    },
        {   
    "field": "260",
    "element": "field",
    "code": None,
    "ind1": None,
    "name": "imprint(260)",
    },
                {   
    "field": "856",
    "element": "subfield",
    "code": "y",
    "ind1": None,
    "name": "files_description(856__y)",
    },
            {   
    "field": "856",
    "element": "subfield",
    "code": "u",
    "ind1": None,
    "name": "files_link(856__u)",
    },
    {
    "field": "710",
    "element": "subfield",
    "code": 'a',
    "ind1": None,
    "name": "corporate_author(710__a)"
    },
        {
    "field": "767",
    "element": "subfield",
    "code": "t",
    "ind1": None,
    "name": "language_title(767__t)"
    },
    {
    "field": "830",
    "element": "subfield",
    "code": "a",
    "ind1": None,
    "name": "series_title(830__a)"
    }
]

### ⏩ 3.2 Convert the XML records into a tabular structure

In [27]:
## RUN AND OBSERVE - Convert the XML into a tabular format and visualize the 5 first rows.
# Print(len(root.findall("record")))
records = md.convert_lxml(xml_tree)
# Print(len(records))
records_dicts = [md.extract_xml(r, record_map) for r in records]
# Print(len(records_dicts))
records_tab = pd.DataFrame(records_dicts)
# Display the first rows
records_tab.head(3)

,undl_id,035__a,isbn(020__a),sales_number(079__a),document_symbol(191__a),title(245),imprint(260),files_description(856__y),files_link(856__u),corporate_author(710__a),language_title(767__t),series_title(830__a)
0,1504,[(DHL)58437],None,[79.IV.1],"[ST/ESA/87, E/CN.5/557]",[{'a': '1978 report on the world social situat...,"[{'a': 'New York :', 'b': 'UN,', 'c': '1 1979.'}]",[English],[http://digitallibrary.un.org/record/1504/file...,[UN. Department for Economic and Social Inform...,None,[World Social Report]
1,72600,"[D177378X, (DHL)177378]",[9211122805],[89.II.D.14],[[TD/]UNCTAD/TDR/9],"[{'a': 'Trade and development report.', 'n': '...","[{'a': 'New York :', 'b': 'UN,', 'c': '1989.'}]","[English, English]",[http://digitallibrary.un.org/record/72600/fil...,[UNCTAD. Secretariat],None,None
2,96615,"[D212161X, (DHL)212161]",[9211122910],[90.II.D.6],[[TD/]UNCTAD/TDR/10],"[{'a': 'Trade and development report.', 'n': '...","[{'a': 'New York :', 'b': 'UN,', 'c': '1990.'}]",[English],[http://digitallibrary.un.org/record/96615/fil...,[UNCTAD. Secretariat],None,None


## 4. Customize the table (Optional)
You can save the CSV with the current structure and go directly to point 3.4. 

### ✅🆎 4.1 Create a new column from a field column

If you extracted a complete field, for instance `991` - you will see the full structure in the cell.
Example with two `991 fields`
```
[{'a': 'A/76/251','b': '38','c': 'The situation in the Middle East.','d': 'MIDDLE EAST SITUATION', 'z': 'I','0': '(DHLAUTH)920146'},{'a': 'S/77','b': '[52]','d': 'ISRAEL--LEBANON','z': 'I','0': '(DHLAUTH)926094'}]
```

Apply the function to create a new column with only the values of a subfield, for instance `d`. 
```
[MIDDLE EAST SITUATION, ISRAEL--LEBANON]
```

Modify the names of columns and the subfield arguments.

In [28]:
# OPTIONAL - UNCOMMENT AND UPDATE
#records_tab['991__d'] = records_tab['991'].apply(md.extract, subfield='d') # Will extract values in subfields d and create a new column
#records_tab.iloc[1]['991__d']

In [29]:
#records_tab.head(2)

### ✅🆎 4.2 Remove subfields in fields columns
If you extracted a complete field, for instance `245`, you will see the full structure in the cell.

`[{'a': 'Draft resolution /', 'c': 'France'}]`

Apply the function `md.flatten()` to remove the subfield structure.

`['Draft resolution / France']`

You can decide to create a new column or to apply the change to the existing column.

In [30]:
# OPTIONAL - UNCOMMENT AND UPDATE
#records_tab['title(245)'] = records_tab['title(245)'].apply(md.flatten)
#records_tab['impring(260)'] = records_tab['impring(260)'].apply(md.flatten)
#records_tab.head(5)

### ✅🆎 4.3 Add links to the metadata editor or/and the UNDL (Optional)
If you have an identifier, and a website that uses the identifer to construct the url. You can use `md.add_link()` function.
Look at the examples for the UNDL and the metadata editor. 

* Metadata editor: https://metadata.un.org/editor/records/bibs/4060945
* UNDL: https://digitallibrary.un.org/record/4060945

Uncomment the relevant line if you wish to add a link to the corresponding records in the system.

In [31]:
# OPTIONAL - UNCOMMENT AND UPDATE - to create link to UNDL form column `undl_id`
#records_tab['undl_link'] = records_tab['undl_id'].apply(md.add_links, template="https://digitallibrary.un.org/record/") 
#records_tab.iloc[1]['undl_link']

In [32]:
# OPTIONAL - UNCOMMENT AND UPDATE to creat link to MARC editor from column 035__a. Change the name of column if not '035__a'
#records_tab['me_link'] = records_tab['035__a'].apply(md.convert_me_id).apply(md.add_links, template="https://metadata.un.org/editor/records/bibs/")
#records_tab.iloc[1]['me_link']

### ✅🆎4.4 Clean the square brackets
To clean the squre brackets at the beginning and end of each cell, apply the function `md.clean()`

In [33]:
# OPTIONAL - UNCOMMENT AND UPDATE
#columns_to_clean = records_tab.columns # will apply the change to all columns
#columns_to_clean = ['710__a', '191__a'] # will apply the change to the column specified only.
#for c in columns_to_clean:
    #records_tab[c] = records_tab[c].apply(md.clean)

In [34]:
records_tab.head(2)

,undl_id,035__a,isbn(020__a),sales_number(079__a),document_symbol(191__a),title(245),imprint(260),files_description(856__y),files_link(856__u),corporate_author(710__a),language_title(767__t),series_title(830__a)
0,1504,[(DHL)58437],None,[79.IV.1],"[ST/ESA/87, E/CN.5/557]",[{'a': '1978 report on the world social situat...,"[{'a': 'New York :', 'b': 'UN,', 'c': '1 1979.'}]",[English],[http://digitallibrary.un.org/record/1504/file...,[UN. Department for Economic and Social Inform...,None,[World Social Report]
1,72600,"[D177378X, (DHL)177378]",[9211122805],[89.II.D.14],[[TD/]UNCTAD/TDR/9],"[{'a': 'Trade and development report.', 'n': '...","[{'a': 'New York :', 'b': 'UN,', 'c': '1989.'}]","[English, English]",[http://digitallibrary.un.org/record/72600/fil...,[UNCTAD. Secretariat],None,None


### ✅🆎4.4 Delete columns
To clean the squre brackets at the beginning and end of each cell, apply the function `md.clean()`

In [41]:
# OPTIONAL - UNCOMMENT AND UPDATE
columns_to_delete = ['035__a']
records_tab.drop(columns=columns_to_delete, inplace=True)
records_tab.head(5)

,undl_id,isbn(020__a),sales_number(079__a),document_symbol(191__a),title(245),imprint(260),files_description(856__y),files_link(856__u),corporate_author(710__a),language_title(767__t),series_title(830__a)
0,1504,None,[79.IV.1],"[ST/ESA/87, E/CN.5/557]",[{'a': '1978 report on the world social situat...,"[{'a': 'New York :', 'b': 'UN,', 'c': '1 1979.'}]",[English],[http://digitallibrary.un.org/record/1504/file...,[UN. Department for Economic and Social Inform...,None,[World Social Report]
1,72600,[9211122805],[89.II.D.14],[[TD/]UNCTAD/TDR/9],"[{'a': 'Trade and development report.', 'n': '...","[{'a': 'New York :', 'b': 'UN,', 'c': '1989.'}]","[English, English]",[http://digitallibrary.un.org/record/72600/fil...,[UNCTAD. Secretariat],None,None
2,96615,[9211122910],[90.II.D.6],[[TD/]UNCTAD/TDR/10],"[{'a': 'Trade and development report.', 'n': '...","[{'a': 'New York :', 'b': 'UN,', 'c': '1990.'}]",[English],[http://digitallibrary.un.org/record/96615/fil...,[UNCTAD. Secretariat],None,None
3,125926,[9211123097],[91.II.D.15],[[TD/]UNCTAD/TDR/11],"[{'a': 'Trade and development report.', 'n': '...","[{'a': 'New York :', 'b': 'UN,', 'c': '1991.'}]",[English],[http://digitallibrary.un.org/record/125926/fi...,[UNCTAD. Secretariat],None,None
4,134374,[9211421594],[90.IX.8],[***],[{'a': 'Science and technology and their impli...,"[{'a': 'New York :', 'b': 'UN,', 'c': '1990.'}]",[English],[http://digitallibrary.un.org/record/134374/fi...,[UN. Department for Disarmament Affairs (1983-...,None,None


## 🆎 5. Save in CSV

🆎 Un-comment the code
🆎 Set the XML file path using the following structure `../folder/file_name.xml`
* `folder`: name of the folder where you store the export (i.e. data)
* `test.csv`: name of the file (i.e. voting_records.xml)


In [42]:
## UPDATE - update the document path
path = '../data/2024_12_13_sales_publication_extract.csv'

In [43]:
records_tab.to_csv(path, index=False, encoding='utf-8')